In [1]:
#Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


#Import dataset
time_series = pd.read_csv('dataset/time_series_after_exploratory_analysis_and_cleaning_data_1.csv', sep = ';', index_col = 0)

In [7]:
#We will use line 0 to 18 to train the model, and try to predict lines 19 and 20
#add new_columns
time_series.loc[:,2016] = 0
time_series.loc[:,2017] = 0

#Predict values

for key,line in time_series.iterrows():
    if line['product'] == 'Livestock':
        poly = PolynomialFeatures(degree = 4)
    else:
        poly = PolynomialFeatures(degree = 2)
    #Adjust the model
    y = list(line.iloc[0:19])
    x = poly.fit_transform(np.array(range(0,19)).reshape(-1,1))
    line = LinearRegression()
    line.fit(x,y)

    #Predict the values
    results = line.predict(poly.fit_transform(np.array([19,20]).reshape(-1,1)))

    #Insert values into dataframe
    time_series.loc[key,2016] = 0 if results[0] < 0 else results[0]
    time_series.loc[key,2017] = 0 if results[1] < 0 else results[1]

In [8]:
time_series

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,city_code,product,product_type,2016,2017
0/Beans/temporary,15.000000,6.000000,5.000000,6.000000,8.000000,8.000000,10.000000,11.856776,15.000000,15.000000,...,25.000000,18.631438,20.000000,15.000000,20.663836,0307883b5d063703,Beans,temporary,22.760844,24.016960
0/Cassava/temporary,235.000000,100.000000,40.000000,100.000000,40.000000,40.000000,60.000000,150.000000,50.000000,100.000000,...,150.000000,116.000000,200.000000,50.000000,116.000000,0307883b5d063703,Cassava,temporary,204.496388,228.029515
0/Corn/temporary,215.000000,100.000000,50.000000,70.000000,70.000000,60.000000,70.000000,83.655786,60.000000,80.000000,...,58.737394,55.968684,50.000000,60.000000,47.662553,0307883b5d063703,Corn,temporary,76.468279,81.997505
0/Livestock/pasture,11349.629010,14146.672050,13548.315410,10631.870570,7243.590422,3527.520999,3440.544659,5208.068994,5874.184988,6700.275067,...,5461.526987,8891.339582,9911.864474,9992.379236,8384.741529,0307883b5d063703,Livestock,pasture,9709.929070,10960.579782
0/Others/permanent,30.000000,25.000000,26.000000,55.000000,54.000000,52.000000,0.000000,0.000000,0.000000,20.000000,...,60.000000,60.000000,63.000000,67.000000,76.000000,0307883b5d063703,Others,permanent,76.872033,88.040660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6/Açaí/permanent,3440.920143,10.000000,2932.019146,3503.158169,3258.674975,3362.584673,3556.267765,10.000000,4545.950143,4020.775884,...,3450.551002,5754.292566,4388.563237,4106.919329,5336.276292,07ce2e79e19977e9,Açaí,permanent,4854.961119,4975.192887
45/Açaí/permanent,40921.893357,44087.669343,47248.861657,42204.700925,49354.384386,50745.829243,52948.113886,49314.026462,56451.071043,50445.752575,...,51025.911550,53464.175775,50270.008925,56331.527171,53519.794087,51db62cb09adc2e8,Açaí,permanent,50090.005528,48724.664665
24/Sorghum/temporary,443.437530,444.108414,449.446238,410.309883,421.303296,384.176311,478.509402,435.485524,341.882049,336.976435,...,394.331470,429.450866,458.793507,392.627283,501.717684,20e241460ce4140d,Sorghum,temporary,454.074447,464.626302
83/Cocoa/permanent,14686.542336,14749.267127,15573.947054,16391.796033,14484.910449,15548.396573,15323.412241,15620.839814,17546.668529,16430.549057,...,16739.095471,16464.895957,17705.687543,18805.622157,16806.442443,86d92d4b0a058014,Cocoa,permanent,16915.401553,16835.553077


In [9]:
#Export in the correct result
part_1 = time_series.iloc[:,21:25]
part_1.loc[:,'year'] = '2016-01-01'
part_1.reset_index(inplace = True)
part_1 = part_1.iloc[:,1:6]
part_1.columns = ['city_code', 'product', 'product_type', 'destinated_area', 'year']
part_2 = time_series.iloc[:,[21,22,23,25]]
part_2.loc[:,'year'] = '2017-01-01'
part_2.reset_index(inplace = True)
part_2 = part_2.iloc[:,1:6]
part_2.columns = ['city_code', 'product', 'product_type', 'destinated_area', 'year']
df = pd.concat([part_1,part_2])
df.to_csv('8GDLJG.csv', sep = ';')

/tmp/ipykernel_274942/3616541508.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part_2.loc[:,'year'] = '2017-01-01'


In [10]:
def wmape(time_series):
    real_part_1 = time_series.iloc[:,19].values
    real_part_2 = time_series.iloc[:,20].values
    real = np.concatenate([real_part_1,real_part_2])
    predict_part_1 = time_series.loc[:,2016].values
    predict_part_2 = time_series.loc[:,2017].values
    predict = np.concatenate([predict_part_1,predict_part_2])
    return abs(real-predict).sum()/real.sum()

In [11]:
#Calculate the error
(wmape(time_series.loc[time_series.product_type == 'pasture']) + \
wmape(time_series.loc[time_series.product_type == 'temporary']) + \
wmape(time_series.loc[time_series.product_type == 'permanent']))/3

0.16000636829538667

In [13]:
wmape(time_series.loc[time_series.product_type == 'pasture'])

0.05140318106875042

In [14]:
wmape(time_series.loc[time_series.product_type == 'temporary'])

0.2593991634417703

In [15]:
wmape(time_series.loc[time_series.product_type == 'permanent'])

0.16921676037563932